In [1]:
from sqlalchemy import create_engine, text

In [2]:
user = "root"
password = "Levp13aa"
host = "localhost"
database = "futbol_dw"

engine = create_engine(f"mysql+pymysql://{user}:{password}@{host}/{database}")

In [3]:
with engine.connect() as conn:
    # === Dimensiones ===
    conn.execute(text("DROP TABLE IF EXISTS dim_jugadores;"))
    conn.execute(text("""
        CREATE TABLE dim_jugadores (
            id_jugador INT PRIMARY KEY,
            nombre VARCHAR(100),
            nacionalidad VARCHAR(50),
            posicion VARCHAR(50),
            edad INT,
            valor_mercado DECIMAL(15,2),
            equipo_actual VARCHAR(100),
            liga VARCHAR(50),
            temporada VARCHAR(20),
            file_source VARCHAR(100)
        );
    """))

    conn.execute(text("DROP TABLE IF EXISTS dim_equipos;"))
    conn.execute(text("""
        CREATE TABLE dim_equipos (
            id_equipo INT AUTO_INCREMENT PRIMARY KEY,
            nombre_equipo VARCHAR(100),
            liga VARCHAR(50),
            temporada VARCHAR(20),
            file_source VARCHAR(100)
        );
    """))

    conn.execute(text("DROP TABLE IF EXISTS dim_partidos;"))
    conn.execute(text("""
        CREATE TABLE dim_partidos (
            id_partido INT AUTO_INCREMENT PRIMARY KEY,
            fecha DATE,
            competencia VARCHAR(80),
            jornada VARCHAR(30),
            venue ENUM('Home','Away'),
            temporada VARCHAR(20),
            liga VARCHAR(50),
            file_source VARCHAR(100)
        );
    """))

    conn.execute(text("DROP TABLE IF EXISTS dim_tiempo;"))
    conn.execute(text("""
        CREATE TABLE dim_tiempo (
            id_tiempo INT AUTO_INCREMENT PRIMARY KEY,
            fecha DATE,
            anio INT,
            mes INT,
            dia INT,
            dia_semana VARCHAR(15),
            temporada VARCHAR(20)
        );
    """))

    # === Tablas de Hechos ===
    conn.execute(text("DROP TABLE IF EXISTS fact_jugador_partido;"))
    conn.execute(text(""" 
        CREATE TABLE fact_jugador_partido (
            id_jugador INT,
            id_equipo INT,
            id_partido INT,
            player_rank INT,
            match_date DATE,
            competition VARCHAR(50),
            player_age VARCHAR(20),
            nation VARCHAR(50),
            is_starter BOOLEAN,
            minutes_played INT,
            goals INT,
            assists INT,
            goals_plus_assists INT,
            goals_non_penalty INT,
            penalty_goals INT,
            penalty_attempts INT,
            penalty_missed INT,
            shots_total INT,
            shots_on_target INT,
            shots_on_target_pct DECIMAL(5,2),
            shot_avg_distance DECIMAL(6,2),
            shots_free_kicks INT,
            goals_per_shot DECIMAL(5,3),
            goals_per_shot_on_target DECIMAL(5,3),
            passes_attempted INT,
            passes_completed INT,
            passes_completion_pct DECIMAL(5,2),
            key_passes INT,
            passes_final_third INT,
            passes_into_penalty_area INT,
            crosses_into_penalty_area INT,
            progressive_passes INT,
            passes_total_distance DECIMAL(10,2),
            passes_progressive_distance DECIMAL(10,2),
            passes_live_shot INT,
            passes_dead_shot INT,
            take_ons_shot INT,
            shots_lead_shot INT,
            fouls_drawn_shot INT,
            defensive_actions_shot INT,
            passes_live_goal INT,
            passes_dead_goal INT,
            take_ons_goal INT,
            shots_lead_goal INT,
            fouls_drawn_goal INT,
            defensive_actions_goal INT,
            touches_total INT,
            touches_def_pen_area INT,
            touches_def_third INT,
            touches_mid_third INT,
            touches_att_third INT,
            touches_att_pen_area INT,
            touches_live INT,
            tackles_total INT,
            tackles_won INT,
            tackles_def_third INT,
            tackles_mid_third INT,
            tackles_att_third INT,
            dribblers_challenged INT,
            dribblers_tackled_pct DECIMAL(5,2),
            dribblers_lost INT,
            blocks_total INT,
            blocks_shot INT,
            blocks_pass INT,
            interceptions INT,
            tackles_plus_interceptions INT,
            clearances INT,
            errors_lead_shot INT,
            yellow_cards INT,
            second_yellow_cards INT,
            red_cards INT,
            fouls_committed INT,
            fouls_drawn INT,
            league VARCHAR(50),
            season VARCHAR(20),
            file_source VARCHAR(100),
            PRIMARY KEY (id_jugador, id_partido)
        );
    """))

    conn.execute(text("DROP TABLE IF EXISTS fact_equipo_partido;"))
    conn.execute(text("""
        CREATE TABLE fact_equipo_partido (
            id_equipo INT,
            id_partido INT,
            team_rank INT,
            match_date DATE,
            competition VARCHAR(50),
            match_round VARCHAR(30),
            venue ENUM('Home','Away'),
            result CHAR(1),
            goals_for INT,
            goals_against INT,
            possession_pct DECIMAL(5,2),
            shots_total INT,
            shots_on_target INT,
            shots_on_target_pct DECIMAL(5,2),
            shot_avg_distance DECIMAL(6,2),
            shots_free_kicks INT,
            penalties_scored INT,
            penalties_attempted INT,
            goals_per_shot DECIMAL(5,3),
            goals_per_shot_on_target DECIMAL(5,3),
            shots_per_90 DECIMAL(5,3),
            shots_on_target_per_90 DECIMAL(5,3),
            goals_per_90 DECIMAL(5,3),
            assists_per_90 DECIMAL(5,3),
            goals_plus_assists_per_90 DECIMAL(5,3),
            expected_goals DECIMAL(6,3),
            non_penalty_xg DECIMAL(6,3),
            expected_assisted_goals DECIMAL(6,3),
            non_penalty_xg_plus_xag DECIMAL(6,3),
            progressive_carries INT,
            progressive_passes INT,
            progressive_receptions INT,
            touches_total INT,
            touches_def_pen_area INT,
            touches_def_third INT,
            touches_mid_third INT,
            touches_att_third INT,
            touches_att_pen_area INT,
            touches_live INT,
            passes_attempted INT,
            passes_completed INT,
            passes_completion_pct DECIMAL(5,2),
            key_passes INT,
            passes_final_third INT,
            passes_into_penalty_area INT,
            crosses_into_penalty_area INT,
            passes_total_distance DECIMAL(10,2),
            passes_progressive_distance DECIMAL(10,2),
            tackles_total INT,
            tackles_won INT,
            blocks_total INT,
            blocks_shot INT,
            blocks_pass INT,
            interceptions INT,
            clearances INT,
            errors_lead_shot INT,
            yellow_cards INT,
            red_cards INT,
            fouls_committed INT,
            fouls_drawn INT,
            league VARCHAR(50),
            season VARCHAR(20),
            file_source VARCHAR(100),
            PRIMARY KEY (id_equipo, id_partido)
        );
    """))

    conn.commit()

print("✅ Tablas de la capa SILVER creadas correctamente.")


✅ Tablas de la capa SILVER creadas correctamente.


# Dimension jugadores 

Vamos a ver como esta en bronze 

In [4]:
import pandas as pd
from sqlalchemy import create_engine

user = "root"
password = "Levp13aa"
host = "localhost"
database = "futbol_dw"

engine = create_engine(f"mysql+pymysql://{user}:{password}@{host}/{database}")

df = pd.read_sql("SELECT * FROM bronze_player_market LIMIT 5;", engine)
print(df.columns.tolist())
print(df.head())

['Nombre', 'Posición', 'Edad', 'Nacionalidad', 'Altura', 'Pie_hábil', 'Contrato', 'Valor_de_mercado', 'Equipo', 'Liga', 'Lesión']
            Nombre     Posición  Edad           Nacionalidad  Altura  \
0      Diant Ramaj   Goalkeeper    24  ['Germany', 'Kosovo']   189.0   
1     Frank Feller   Goalkeeper    21            ['Germany']   183.0   
2     Kevin Müller   Goalkeeper    34            ['Germany']   190.0   
3  Paul Tschernuth   Goalkeeper    23            ['Austria']   186.0   
4  Benedikt Gimber  Centre-Back    28            ['Germany']   187.0   

  Pie_hábil    Contrato  Valor_de_mercado              Equipo        Liga  \
0     right  2026-06-30         7000000.0  1.FCHeidenheim1846  Bundesliga   
1     right  2029-06-30          500000.0  1.FCHeidenheim1846  Bundesliga   
2     right  2027-06-30          400000.0  1.FCHeidenheim1846  Bundesliga   
3      left  2027-06-30          100000.0  1.FCHeidenheim1846  Bundesliga   
4      left  2028-06-30         3000000.0  1.FCHeide

In [5]:
# === 2. Leer tabla desde Bronze ===
df = pd.read_sql("SELECT * FROM bronze_player_market;", engine)

# === 3. Normalizar nombres ===
df.columns = [c.lower().strip().replace(" ", "_") for c in df.columns]

# Confirmamos nombres
print(df.columns.tolist())


['nombre', 'posición', 'edad', 'nacionalidad', 'altura', 'pie_hábil', 'contrato', 'valor_de_mercado', 'equipo', 'liga', 'lesión']


In [6]:
# === 4. Seleccionar y renombrar ===
df = df.rename(columns={
    'nombre': 'nombre',
    'posición': 'posicion',
    'edad': 'edad',
    'nacionalidad': 'nacionalidad',
    'altura': 'altura',
    'pie_hábil': 'pie_habil',
    'contrato': 'contrato_fin',
    'valor_de_mercado': 'valor_mercado',
    'equipo': 'equipo_actual',
    'liga': 'liga',
    'lesión': 'lesion'
})

In [7]:
# === 5. Limpieza ===
df['nombre'] = df['nombre'].str.strip().str.lower()
df['equipo_actual'] = df['equipo_actual'].str.strip().str.lower()
df['liga'] = df['liga'].str.strip()
df['valor_mercado'] = pd.to_numeric(df['valor_mercado'], errors='coerce')
df['altura'] = pd.to_numeric(df['altura'], errors='coerce')

# Normalizar nacionalidad (lista → texto)
df['nacionalidad'] = df['nacionalidad'].astype(str).str.replace(r"[\[\]']", "", regex=True)

# === 6. Añadir metadatos ===
df['temporada'] = '2024-2025'
df['file_source'] = 'Transfermrkt.csv'
df.insert(0, 'id_jugador', range(1, len(df) + 1))  # genera un ID secuencial

In [8]:
# === 7. Subir a MySQL (Silver) ===
df.to_sql("dim_jugadores", engine, if_exists="replace", index=False, method="multi", chunksize=1000)

print(f" Tabla 'dim_jugadores' creada correctamente con {len(df)} registros y {len(df.columns)} columnas.")

 Tabla 'dim_jugadores' creada correctamente con 2600 registros y 14 columnas.


In [10]:
df = pd.read_sql("SELECT * FROM dim_jugadores LIMIT 5;", engine)
df.head()

,id_jugador,nombre,posicion,edad,nacionalidad,altura,pie_habil,contrato_fin,valor_mercado,equipo_actual,liga,lesion,temporada,file_source
0,1,diant ramaj,Goalkeeper,24,"Germany, Kosovo",189.0,right,2026-06-30,7000000.0,1.fcheidenheim1846,Bundesliga,Sin lesión,2024-2025,Transfermrkt.csv
1,2,frank feller,Goalkeeper,21,Germany,183.0,right,2029-06-30,500000.0,1.fcheidenheim1846,Bundesliga,Torn lateral knee ligament - Return unknown,2024-2025,Transfermrkt.csv
2,3,kevin müller,Goalkeeper,34,Germany,190.0,right,2027-06-30,400000.0,1.fcheidenheim1846,Bundesliga,Sin lesión,2024-2025,Transfermrkt.csv
3,4,paul tschernuth,Goalkeeper,23,Austria,186.0,left,2027-06-30,100000.0,1.fcheidenheim1846,Bundesliga,Sin lesión,2024-2025,Transfermrkt.csv
4,5,benedikt gimber,Centre-Back,28,Germany,187.0,left,2028-06-30,3000000.0,1.fcheidenheim1846,Bundesliga,Sin lesión,2024-2025,Transfermrkt.csv


# Dimension partidos 

# Dimension equipos 